In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',500)

calendar_dfs = []
listings_dfs = []
reviews_dfs = []

for subdir, dirs, files in  os.walk("./seattle_data/"):
    for file in files:
        pth = os.path.join(subdir,file)
        if "calendar" in file:
            c_data = pd.read_csv(pth)
            calendar_dfs.append(c_data)
        elif "listings" in file:
            l_data = pd.read_csv(pth)
            listings_dfs.append(l_data)
        elif "reviews" in file:
            r_data = pd.read_csv(pth)
            reviews_dfs.append(r_data)

calendar_df = pd.concat(calendar_dfs,ignore_index=True)
listings_df = pd.concat(listings_dfs,ignore_index=True)
reviews_df = pd.concat(reviews_dfs,ignore_index=True)

In [5]:
cal_df = calendar_df.copy()
lst_df = listings_df.copy()
rev_df = reviews_df.copy()

cal_df.dropna(inplace=True)
# cal_df.drop(labels=['adjusted_price'],axis=1,inplace=True)
cal_df['date'] = pd.to_datetime(calendar_df['date'])
cal_df['price'] = cal_df['price'].str.slice(start=1)
cal_df['price'] = cal_df['price'].replace(',','',regex=True)
cal_df['price'] = pd.to_numeric(cal_df['price'], errors='coerce')
# Do not filter for only available: cal_df = cal_df[cal_df['available'] == 't']
cal_df.drop(labels=['available'],axis=1,inplace=True)
cal_df.head()

,listing_id,date,price
0,241032,2016-01-04,85.0
1,241032,2016-01-05,85.0
9,241032,2016-01-13,85.0
10,241032,2016-01-14,85.0
14,241032,2016-01-18,85.0


In [7]:
lst_df.dtypes

id                                    int64
listing_url                          object
scrape_id                             int64
last_scraped                         object
name                                 object
summary                              object
space                                object
description                          object
experiences_offered                  object
neighborhood_overview                object
notes                                object
transit                              object
thumbnail_url                        object
medium_url                           object
picture_url                          object
xl_picture_url                       object
host_id                               int64
host_url                             object
host_name                            object
host_since                           object
host_location                        object
host_about                           object
host_response_time              

In [ ]:
# Avoid features specific to market like neighborhood
features = ['space','experiences_offered','transit','host_since',\
            'host_location','host_response_time','host_response_rate',\
            'host_acceptance_rate','host_listings_count',\
            'host_total_listings_count','host_verifications',\
            'host_has_profile_pic','host_identity_verified',\
            'property_type','room_type','accommodates','bathrooms'\
            'bedrooms','beds','bed_type','amenities','square_feet',\
            'price','security_deposit','cleaning_fee','minimum_nights','maximum_nights',\
             'availability_365','number_of_reviews','last_review','','',\
              '','','','','',\
               '','','','','',\
                '','','','','',\
                 '','','','','',\
                  '','','','','',\
            ]
target = ['host_is_superhost']

for col in features:
    if lst_df[col].dtype == 'object':
        print(lst_df[col].value_counts())
        print(lst_df[col].isna().sum())
        # Ordinal/Categorical ... to_numeric()
        # If date maybe change to is_last_30, is_last_90?
